# How to create your pipeline step-by-step

In this tutorial, you'll explore **Picanteo** framework and learn how to design and implement your own pipeline.  
In order to cover a large number of modules, we'll be building a bi-temporal optical change detection pipeline integrating 2d and 3d data for this demonstration. 

During this tutorial, you will  discover how to use **Picanteo** to:
- **Extract data**: generate a semantic segmentation map and a Digital Surface Model (DSM)
- **Filter the results**: remove outliers bby applying uncertainty, morphologic and altimetric filters
- **Merge data**: combine the 2D and 3D change map resulting from the previous steps in a single change map    

## **Picanteo** main concepts

First of all, we'll need to explain a few basics about how **Picanteo** works, and in particular the two major concepts: pipelines and steps.

### **Picanteo** pipelines

**Picanteo** is a framework that allows you to launch pre-defined pipelines or create your own. A pipeline is a workflow composed of several steps that transform the input data into enhance results.
Pipelines are Python classes that define all the steps required to produce the targeted result. They are launched using a configuration file in Yaml format.
This configuration file contains not only the paths to the input data, but also the links to the configuration files used by each step in the pipeline.


<div align="left">
    <img src="images/picanteo_pipeline.png">
</div>  

Here is a example of a pipeline configuration file content:
```yaml
# pipeline (str): describe the pipeline to run    
pipeline: my_pipeline
# pipeline_output_dir (str|Path): path to the step output directory    
pipeline_output_dir: my_pipeline_outdir
# save_intermediate_data (bool, optional): if True, retains intermediate files generated during the pipeline execution. 
save_intermediate_data: True

### Pipeline steps configuration ###
# <step>_conf (str|Path): Path to the <step> yaml configuration file template
inference_conf: "conf/steps/my_1st_step_conf.yaml"
inference_conf: "conf/steps/my_2nd_step_conf.yaml"

### Input data ###
# my_img (dict): Path to my input image    
my_img: "data/my_image.tif"
```
> If you want to be able to run your custom pipeline with the CLI, you must add your pipeline name and associated class in `pipeline_types` dict of the `create_pipeline` method in [main_pipeline.py](https://github.com/CNES/picanteo/tree/master/picanteo/main_pipeline.py)

### **Picanteo** steps

A **Picanteo** step defines a module that performs a task. It's the elementary component of the framework and can take many forms: application wrapper, filters, etc.  
Each step has a constructor that takes a yaml configuration file as input, and a run function that launches processing. 
The input yaml config file must contain at least the `step_output_dir` key, which defines the path to the output folder. But it can also optionally contain the optionnal keys: `save_intermediate_data` to retain intermediate files generated during the step execution if `True` (defaults to `False`) and `create_logdir` boolean (defaults to `False`) that indicates if the step should creates a `logs` directory for the step in addition to regular **Picanteo** logger (required if the step produces it's own logs).

Each step writes its result on disk. There's no notion of streaming for the moment. I/O accesses may affect runtime, but this architecture makes it very easy to link steps together in a modular approach.
Furthermore, all intermediate products can be retrieved for visualization or debugging.

### **Picanteo** toolbox

**Picanteo** already has a wide range of steps available in the toolbox (`picanteo/toolbox`). 
They are categorized according to their usage:
- **data_extraction**: package of steps used to generate data from user inputs (3D models, land use map, etc.)
- **data_filtering**: set of filters that enhance raw products from *data_extraction* steps
- **data_fusion**: steps for merging different modalities into a single one
- **data_visualization**: steps defining the output dashboard

For each step defined in the toolbox, you can find its associated configuration file in the [`conf`](https://github.com/CNES/picanteo/tree/master/conf) folder.  

If you can't find the functionality you need, you can create your own module by inheriting from the `PicanteoStep` ([picanteo_step.py](https://github.com/CNES/picanteo/tree/master/picanteo/toolbox/picanteo_step.py)) class:
```Python
class DsmExtraction(PicanteoStep):
    # Your stuff here
``` 

Each step must implement these three functions:
- `run`: the step core, i.e. the function describing the task to perform
- `clean`: if the step produce intermediate files, this function must remove those element if the step parameter `save_intermediate_data` is `False` (the default value)
- `store_log`: move step-specific logs to the log directory if the application(s) run by the step produce it's own log in addition to the **Picanteo** logger

## Installation

### Download software 

For this tutorial we will install **Picanteo** with pip:  
*Alternatively, you can install via Github (```git clone https://github.com/CNES/picanteo.git```) if you want to run your pipeline with the CLI since your required the source code of **Picanteo**.* 

In [ ]:
!python --version

In [ ]:
!pip install picanteo

In [ ]:
!picanteo -h

### Download tutorial input data

In this tutorial, we'll be working on a real earthquake scenario.  
In February 2023, Turkey and Syria experienced devastating earthquakes, we'll use two Pleiades stereo acquisitions over the city of Kahramanmaraş area in Turkey and try to detect the destroyed buildings.  
The pre-event image was acquired on 23/04/2016 and the post-event image was taken on 15/04/2023.  
We cropped the images to focus on a tiny area.

In [ ]:
import os
from matplotlib import pyplot
import rasterio
from rasterio.plot import show

!mkdir -p data
for pair in ["pre", "post"]:
    for mod in ["img", "color"]:
        for idx in range(2):
            for ext in [".geom", ".tif"]:
                filename = pair+"_event_"+mod+str(idx+1)+ext
                if os.path.exists(filename) is False:
                    filename = "https://raw.githubusercontent.com/CNES/picanteo/main/docs/tutorials/data/"+filename
                    !wget {filename} -P data/

In [ ]:
fig, axs = pyplot.subplots(2, 2, figsize=(10,10))

for idx1, pair in enumerate(["pre", "post"]):
    for idx2 in range(2):
        with rasterio.open("data/"+pair+"_event_img"+str(idx2+1)+".tif") as src:
            show((src), ax=axs[idx1, idx2], cmap='gray', title=pair+"-event "+str(idx2+1))

### (Optional) Use your own data

If you want to use your own data, you have to pre-process them in order to run this tutorial. You will have to apply geometric and radiometric corrections to your input data.  
All the process is describe in this [tutorial](https://github.com/CNES/picanteo/tree/master/docs/tutorials/data_readiness.md).

## Let's create our pipeline!

- define how it will works (illustration)
- update the factory
- add step one-by-one and show the result each time

*You can find the final file of this tutorial [here](https://github.com/CNES/picanteo/docs/tutorials/how_to_create_a_pipeline.py).*